# Visualización Interactiva con Plotly

Este notebook demuestra cómo usar **Plotly** para crear gráficos interactivos en PetroKit.

**Fase 3 - Profesionalización**: Ejemplos con gráficos interactivos

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from petrokit.ipr import ipr_curve_vogel, ipr_curve_fetkovich, ipr_curve_jones, ipr_curve_standing
from petrokit.vlp import vlp_curve
from petrokit.nodal import nodal_analysis

## 1. Análisis Nodal Interactivo Básico

Gráfico interactivo con zoom, pan, y hover para ver valores exactos.

In [ ]:
# Parámetros
p_res = 3000
q_max = 1200
well_depth = 8000
rho = 60
mu = 1.0
d = 2.992

# Calcular curvas
pwf_ipr, q_ipr = ipr_curve_vogel(p_res, q_max)
q_vlp = np.linspace(0, 1200, 50)
pwf_vlp = vlp_curve(q_vlp, well_depth, rho, mu, d)

# Calcular punto de operación
q_op, pwf_op = nodal_analysis(p_res, q_max, well_depth, rho, mu, d)

# Crear figura interactiva
fig = go.Figure()

# Añadir IPR
fig.add_trace(go.Scatter(
    x=q_ipr, y=pwf_ipr,
    mode='lines',
    name='IPR (Vogel)',
    line=dict(color='blue', width=3),
    hovertemplate='<b>IPR</b><br>Q: %{x:.1f} STB/d<br>Pwf: %{y:.1f} psi<extra></extra>'
))

# Añadir VLP
fig.add_trace(go.Scatter(
    x=q_vlp, y=pwf_vlp,
    mode='lines',
    name='VLP',
    line=dict(color='red', width=3),
    hovertemplate='<b>VLP</b><br>Q: %{x:.1f} STB/d<br>Pwf: %{y:.1f} psi<extra></extra>'
))

# Añadir punto de operación
if q_op > 0:
    fig.add_trace(go.Scatter(
        x=[q_op], y=[pwf_op],
        mode='markers',
        name='Punto de Operación',
        marker=dict(color='green', size=15, symbol='circle'),
        hovertemplate=f'<b>Punto de Operación</b><br>Q: {q_op:.1f} STB/d<br>Pwf: {pwf_op:.1f} psi<extra></extra>'
    ))

# Configurar layout
fig.update_layout(
    title=dict(
        text='Análisis Nodal Interactivo<br><sub>Haz zoom, pan, o pasa el mouse sobre las curvas</sub>',
        x=0.5,
        xanchor='center'
    ),
    xaxis=dict(
        title='Caudal (STB/d)',
        gridcolor='lightgray',
        showgrid=True
    ),
    yaxis=dict(
        title='Presión de Fondo (psi)',
        gridcolor='lightgray',
        showgrid=True
    ),
    hovermode='x unified',
    template='plotly_white',
    width=900,
    height=600,
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()

## 2. Comparación de Múltiples Escenarios

Compara diferentes diámetros de tubing en una sola gráfica interactiva.

In [ ]:
# Parámetros base
p_res = 3000
q_max = 1200
well_depth = 8000
rho = 60
mu = 1.0

# Diferentes diámetros
diameters = [2.0, 2.5, 2.992, 3.5, 4.0]
colors = ['rgb(255,0,0)', 'rgb(255,127,0)', 'rgb(0,255,0)', 'rgb(0,0,255)', 'rgb(127,0,255)']

# IPR (única)
pwf_ipr, q_ipr = ipr_curve_vogel(p_res, q_max)

# Crear figura
fig = go.Figure()

# Añadir IPR
fig.add_trace(go.Scatter(
    x=q_ipr, y=pwf_ipr,
    mode='lines',
    name='IPR (Vogel)',
    line=dict(color='black', width=3, dash='dot'),
    hovertemplate='<b>IPR</b><br>Q: %{x:.1f} STB/d<br>Pwf: %{y:.1f} psi<extra></extra>'
))

# Añadir VLP para cada diámetro
q_vlp = np.linspace(0, 1200, 50)
for i, d in enumerate(diameters):
    pwf_vlp = vlp_curve(q_vlp, well_depth, rho, mu, d)
    q_op, pwf_op = nodal_analysis(p_res, q_max, well_depth, rho, mu, d)
    
    # VLP
    fig.add_trace(go.Scatter(
        x=q_vlp, y=pwf_vlp,
        mode='lines',
        name=f'VLP (d={d}")',
        line=dict(color=colors[i], width=2),
        hovertemplate=f'<b>VLP (d={d}")</b><br>Q: %{{x:.1f}} STB/d<br>Pwf: %{{y:.1f}} psi<extra></extra>'
    ))
    
    # Punto de operación
    if q_op > 0:
        fig.add_trace(go.Scatter(
            x=[q_op], y=[pwf_op],
            mode='markers',
            name=f'Op. d={d}" ({q_op:.0f} STB/d)',
            marker=dict(color=colors[i], size=10, symbol='circle'),
            hovertemplate=f'<b>Op. d={d}"</b><br>Q: {q_op:.1f} STB/d<br>Pwf: {pwf_op:.1f} psi<extra></extra>'
        ))

# Layout
fig.update_layout(
    title=dict(
        text='Análisis de Sensibilidad: Efecto del Diámetro de Tubing<br><sub>Comparación de múltiples escenarios</sub>',
        x=0.5,
        xanchor='center'
    ),
    xaxis=dict(title='Caudal (STB/d)', gridcolor='lightgray'),
    yaxis=dict(title='Presión de Fondo (psi)', gridcolor='lightgray'),
    hovermode='closest',
    template='plotly_white',
    width=1000,
    height=650,
    showlegend=True,
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)

fig.show()

## 3. Comparación de Modelos IPR

Visualiza diferentes modelos IPR en una sola gráfica interactiva.

In [ ]:
# Parámetros
p_res = 3000
q_max = 1200
J = 0.5
C = 0.5
D = 0.001
p_b = 2200

# Calcular diferentes IPRs
pwf_vogel, q_vogel = ipr_curve_vogel(p_res, q_max)
pwf_fetkovich, q_fetkovich = ipr_curve_fetkovich(p_res, J)
pwf_jones, q_jones = ipr_curve_jones(p_res, C, D)
pwf_standing, q_standing = ipr_curve_standing(p_res, p_b, J)

# Crear figura
fig = go.Figure()

# Añadir cada modelo
models = [
    ('Vogel', q_vogel, pwf_vogel, 'blue'),
    ('Fetkovich', q_fetkovich, pwf_fetkovich, 'red'),
    ('Jones', q_jones, pwf_jones, 'green'),
    ('Standing', q_standing, pwf_standing, 'purple')
]

for name, q, pwf, color in models:
    fig.add_trace(go.Scatter(
        x=q, y=pwf,
        mode='lines',
        name=f'IPR - {name}',
        line=dict(width=3),
        hovertemplate=f'<b>{name}</b><br>Q: %{{x:.1f}} STB/d<br>Pwf: %{{y:.1f}} psi<extra></extra>'
    ))

# Layout
fig.update_layout(
    title=dict(
        text='Comparación de Modelos IPR<br><sub>Vogel vs Fetkovich vs Jones vs Standing</sub>',
        x=0.5,
        xanchor='center'
    ),
    xaxis=dict(title='Caudal (STB/d)', gridcolor='lightgray'),
    yaxis=dict(title='Presión de Fondo (psi)', gridcolor='lightgray'),
    hovermode='x unified',
    template='plotly_white',
    width=900,
    height=600,
    showlegend=True,
    legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99)
)

fig.show()

## 4. Dashboard Multi-Panel

Crea un dashboard con múltiples subplots para análisis completo.

In [ ]:
# Crear subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('IPR - Vogel', 'VLP - Diferentes diámetros', 
                    'Análisis Nodal', 'Sensibilidad q_max'),
    vertical_spacing=0.12,
    horizontal_spacing=0.10
)

# Panel 1: IPR
pwf_ipr, q_ipr = ipr_curve_vogel(3000, 1200)
fig.add_trace(go.Scatter(x=q_ipr, y=pwf_ipr, name='IPR', line=dict(color='blue')), row=1, col=1)

# Panel 2: VLP con diferentes diámetros
q_vlp = np.linspace(0, 1200, 50)
for d, color in [(2.5, 'red'), (2.992, 'green'), (3.5, 'blue')]:
    pwf = vlp_curve(q_vlp, 8000, 60, 1.0, d)
    fig.add_trace(go.Scatter(x=q_vlp, y=pwf, name=f'd={d}"', line=dict(color=color)), row=1, col=2)

# Panel 3: Análisis Nodal
pwf_ipr, q_ipr = ipr_curve_vogel(3000, 1200)
pwf_vlp = vlp_curve(q_vlp, 8000, 60, 1.0, 2.992)
q_op, pwf_op = nodal_analysis(3000, 1200, 8000, 60, 1.0, 2.992)
fig.add_trace(go.Scatter(x=q_ipr, y=pwf_ipr, name='IPR', line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=q_vlp, y=pwf_vlp, name='VLP', line=dict(color='red')), row=2, col=1)
if q_op > 0:
    fig.add_trace(go.Scatter(x=[q_op], y=[pwf_op], name='Op.', mode='markers', 
                            marker=dict(size=10, color='green')), row=2, col=1)

# Panel 4: Sensibilidad q_max
for q_max, color in [(800, 'red'), (1200, 'green'), (1600, 'blue')]:
    pwf, q = ipr_curve_vogel(3000, q_max)
    fig.add_trace(go.Scatter(x=q, y=pwf, name=f'q_max={q_max}', line=dict(color=color)), row=2, col=2)

# Actualizar ejes
fig.update_xaxes(title_text="Caudal (STB/d)", row=1, col=1)
fig.update_xaxes(title_text="Caudal (STB/d)", row=1, col=2)
fig.update_xaxes(title_text="Caudal (STB/d)", row=2, col=1)
fig.update_xaxes(title_text="Caudal (STB/d)", row=2, col=2)

fig.update_yaxes(title_text="Pwf (psi)", row=1, col=1)
fig.update_yaxes(title_text="Pwf (psi)", row=1, col=2)
fig.update_yaxes(title_text="Pwf (psi)", row=2, col=1)
fig.update_yaxes(title_text="Pwf (psi)", row=2, col=2)

# Layout general
fig.update_layout(
    title_text="Dashboard PetroKit - Análisis Integrado",
    showlegend=False,
    height=800,
    width=1200,
    template='plotly_white'
)

fig.show()

## Conclusiones

Este notebook demuestra las capacidades de **Plotly** para crear:

1. ✅ Gráficos interactivos con zoom, pan y hover
2. ✅ Comparaciones de múltiples escenarios
3. ✅ Visualización de diferentes modelos
4. ✅ Dashboards multi-panel para análisis integrado

**Ventajas de Plotly:**
- Interactividad sin necesidad de configuración adicional
- Exportación a HTML para compartir
- Integración perfecta con Jupyter notebooks
- Apariencia profesional lista para presentaciones

**Fase 3 completada**: Ejemplos con gráficos interactivos Plotly ✓